# Visualisation

FILES

https://raw.githubusercontent.com/ScryptIQ-ai/lesson-data/refs/heads/main/dh/EEG_absence.txt
https://raw.githubusercontent.com/ScryptIQ-ai/lesson-data/refs/heads/main/dh/EEG_background.txt

PACKAGES

pandas
matplotlib
numpy

## Introduction

**Learning Objectives**
- How is time series data visualised?
- Making a custom plotting function.

Time series data is unique in that the x-axis is shared (time), however, you do not want to have your individual time series overlapping as this will obscure the data to hand (if you have a lot of time series ). To this extent we will need to extend the use of matplotlib to create data that is offset, rather than overlaid. Creating custom plotting functions is a common task in data analysis and visualisation, and once created you can use them again and again. 

## The Plotting Function

VIDEO

https://www.youtube.com/watch?v=qWYBBXR8Yrs

If you watch the video above you will see the limitations of using Matplotlib in its standard way to plot time series data. To rectify this we have created a plotting function that handles multiple time series data, creating a plot of all the columns in our data, laid out one above the other.

If you are working locally in a Jupyter Notebook, please execute this cell first to ensure the subsequent code runs without error

In [ ]:
# Imports for this lesson
from pandas import read_csv
from numpy import linspace, arange, zeros 
from matplotlib.pyplot import subplots, show, legend

In [ ]:
def plot_series(data, sr):
    '''
    Time series plot of multiple time series
    Data are normalised to mean=0 and var=1
    
    Parameters:
    -----------
    data : numpy.ndarray
        nxm numpy array. Rows are time points, columns are channels
    sr : int
        Sampling rate, same time units as period
    
    Features:
    ---------
    - Normalises each channel to mean=0, std=1
    - Offsets channels vertically for clear separation
    - Adds reference lines at zero for each channel
    - Proper time axis scaling
    - Channel labelling
    '''
    # no. of rows or sampling events
    samples = data.shape[0]
    # no. of columns or sensors taking readings
    sensors = data.shape[1]
    # Length of time series in time
    period = samples // sr
    # Array of all time points
    time = linspace(0, period, period*sr)
    
    offset = 5  # for mean=0 and var=1 normalised data
    
    # Calculate means and standard deviations of all columns
    means = data.mean(axis=0)
    stds = data.std(axis=0)
    
    # Plot each series with an offset of 2 times the standard deviations
    fig, ax = subplots(figsize=(7, 8))
    
    # Plot normalised data with vertical offsets
    ax.plot(time, (data - means)/stds + offset*arange(sensors-1, -1, -1))
    
    # Add reference lines at zero for each channel
    ax.plot(time, zeros((samples, sensors)) + offset*arange(sensors-1, -1, -1),
            '--', color='gray', alpha=0.5)
    
    ax.set(xlabel='Time (seconds)', ylabel='Channel')
    ax.set_yticks(offset*arange(sensors))
    ax.set_yticklabels(arange(sensors)[::-1])
    ax.set_title('Multi-Channel Time Series Plot')
    
    return fig, ax

VIDEO

https://www.youtube.com/watch?v=T2Qh7yjHC9M

## Normal and Pathological EEG

As an example, let us import two sets of time series data and convert these into NumPy arrays, that we will call `data_back` and `data_epil`. They represent a human electroencephalogram (EEG), as recorded during normal background activity and during an epileptic seizure, referred to as an absence seizure.

The `read_csv` function is called in combination with the keyword argument `sep`. When its value is set to `"\s+"`, this allows the user to import data that are space-separated (rather than comma-separated). If you eyeball the contents of the data .txt files, you will notice that the numbers (which represent voltages) are indeed separated by spaces, not commas. As `\` is an escape sequence we need to place an `r` before it to have it treated as a character.

The data is then converted from a Pandas DataFrame into a NumPy array.

Next, three constants are assigned: The sampling rate (sr) is given in the number of samples recorded per second; the duration of the recording (period) which is given in seconds; and the number of columns (channels), to be extracted from the recording. Let's make use of the first 10 columns of data for the remainder of this lesson.

In [ ]:
df_back = read_csv("./data/EEG_background.txt", sep=r"\s+")
df_epil = read_csv("./data/EEG_absence.txt", sep=r"\s+")

d1 = df_back.to_numpy()
d2 = df_epil.to_numpy()

sr = 256      # 1 / seconds
period = 6    # seconds
channels = 10

Now we can filter the NumPy array to only have the first 6 seconds worth of data from the first 10 columns:

In [ ]:
data_back = d1[:period*sr:, :channels]
data_epil = d2[:period*sr:, :channels]

To see the names of the channels (or recording sensors), we can use the `.head()` method, as follows:

In [ ]:
df_back.head()

Lets also check the shape of both datasets.

In [ ]:
print(data_back.shape, data_epil.shape)

The row indices and column names for the seizure data look the same. The names of the recording channels are from the commonly used 10-20 system used to record voltages of brain activity from the human scalp. As an example, 'F' stands for the frontal lobe.

Next, we can call and make use of the plot function that we defined previously, in order to plot the data. If we examine the code in which this function was defined, you can see that we have set it to require the user to specify two input arguments: these correspond to the dataset (data) and sampling rate (sr).

In [ ]:
plot_series(data_back, sr)

show()

In [ ]:
plot_series(data_epil, sr)

show()

## Display data with offset

Scroll up, and take a look at the code we gave previously, that defines the function `plot_series`. Again, this is the function we are using to create our time series plot. It requires the input of a data file where the row index is interpreted as time. In addition, the sampling rate (sr) is required in order to extract the time scale. The sampling rate specifies the number of samples recorded per unit time.

The sensors, or recording channels, are assumed to be in the columns.

The declaration syntax `def` is followed by the function name and, in parentheses, the input arguments. This line ends with a colon, which is required by Python in order to define the start of a block of code.

Following the declaration line, the function's documentation or docstring is contained within two lines of triple backticks. This explains the function's operation, arguments and use – and can contain any other useful information pertaining to the operation of the defined function.

Following the docstring, the main lines of code that operate on the arguments provided by the user, when the function is called.

The function can then be closed using the optional output syntax `return` and any number of returned variables, anything that might be used as a product of running the function.

In our example, the figure environment and the coordinate system are 'returned' and can, in principle, be used to further modify the plot.

The code below illustrates how to call the function and then add a title and the sensor names to the displayed output:

In [ ]:
(fig, ax) = plot_series(data_epil, sr)

names = df_back.columns[:channels]

fig.suptitle('Recording of Absence Seizure', fontsize=16)
legend(names)

show()

The variables passed into a function are called its **input arguments**; and the values it returns are termed its **outputs**. A function ordinarily accepts data or variables in the form of one or several such input arguments, processes these, and subsequently produces a specific output.

A reminder of some of the key points about functions from Python Fundamentals:
- The name of a function follows same principles as that of any other variable. It must be in lower-case characters, and it is strongly suggested that its name bears resemblance to the processes it undertakes.
- The input arguments of a function, e.g. `data` and `sr` in our example, are essentially variables whose scope is confined only to the function. That is, they are only accessible within the function itself, and not from outside the function.
- Variables defined inside of a function should not use the same name as variables defined outside. Otherwise they may override each other.
- When defining a function, it is important and best practice to write that function to perform only one specific task. As such, it can be used independent of the current context. Try to avoid incorporating separable tasks into a single function.
- Once you start creating functions for different purposes you can start to build your own library of ready-to-use functions. This is the primary principle of a popular programming paradigm known as functional programming.

## Summary

By learning to create custom plotting functions that normalise and vertically offset multi-channel time series data, we can effectively visualize complex physiological recordings where standard plotting approaches fall short. Understanding how to structure functions with clear input arguments, proper documentation and reusable outputs enables the development of a personal library of analytical tools tailored to specific research needs. This approach transforms overlapping, difficult-to-interpret traces into clear, separated channels that reveal patterns in datasets such as EEG recordings. Mastering these visualisation techniques facilitates rapid identification of pathological events, comparison across conditions and efficient exploration of large multi-sensor datasets.